# CONTENT BASED FILTERING
## MULTIPLE USERS

In [1]:
# !pip install scikit-surprise


In [10]:
import pandas as pd 
import numpy as np 

from surprise import SVD


In [11]:
movies = ["Terminator 2","Interstellar","Ant Man 2","3 Idiots"]
action = [1,0,1,0]
scifi = [1,1,1,0]
adventure = [0,1,1,0]
comedy = [0,0,1,1]
drama = [0,1,0,1]

df_item_features = pd.DataFrame({
    'movie':movies,
    'Action':action,
    'Sci-Fi':scifi,
    'Adventure':adventure,
    'Comedy':comedy,
    'Drama':drama
})

df_item_features


,movie,Action,Sci-Fi,Adventure,Comedy,Drama
0,Terminator 2,1,1,0,0,0
1,Interstellar,0,1,1,0,1
2,Ant Man 2,1,1,1,1,0
3,3 Idiots,0,0,0,1,1


In [12]:
user = ['user 1','user 2','user 3','user 4']
terminator_2 = [7,8,9,0]
interstellar = [9,0,0,7]
ant_man_2 = [8,6,0,0]
three_idiots = [9,5,10,9]

df_user_items = pd.DataFrame({
    'user':user,
    'Terminator 2':terminator_2,
    'Interstellar':interstellar,
    'Ant Man 2':ant_man_2,
    '3 Idiots':three_idiots
})

df_user_items

,user,Terminator 2,Interstellar,Ant Man 2,3 Idiots
0,user 1,7,9,8,9
1,user 2,8,0,6,5
2,user 3,9,0,0,10
3,user 4,0,7,0,9


In [13]:
arr_user_items = np.array(df_user_items.drop('user', axis = 1))
arr_item_features = np.array(df_item_features.drop('movie', axis = 1))

n_user = arr_user_items.shape[0]
n_item = arr_user_items.shape[1]
n_feature = arr_item_features.shape[1]

arr_user_items_score = np.empty((n_user,n_item))
arr_user_feature = np.empty((n_user,n_feature))

for i in range(0,n_user):
  # print(arr_user_items[i,:])
  user_feature = np.matmul(arr_user_items[i,:],arr_item_features)
  # print(user_feature)
  user_feature = user_feature/user_feature.sum()
  arr_user_feature[i,:] = user_feature



In [14]:
df_user_feature = pd.DataFrame(arr_user_feature)
df_user_feature.columns = df_item_features.columns[1:]
df_user_feature.index = user
df_user_feature

,Action,Sci-Fi,Adventure,Comedy,Drama
user 1,0.164835,0.263736,0.186813,0.186813,0.197802
user 2,0.280000,0.280000,0.120000,0.220000,0.100000
user 3,0.236842,0.236842,0.000000,0.263158,0.263158
user 4,0.000000,0.179487,0.179487,0.230769,0.410256


In [15]:
for i in range(0, n_user):
  user_item_score = np.matmul(arr_item_features,arr_user_feature[i,:])
  arr_user_items_score[i,:] = user_item_score

arr_user_items_score_unwatched = np.where(arr_user_items == 0,arr_user_items_score,0)

df_user_items_score_unwatched = pd.DataFrame(arr_user_items_score_unwatched)
df_user_items_score_unwatched.columns = movies
df_user_items_score_unwatched.index = user
df_user_items_score_unwatched


,Terminator 2,Interstellar,Ant Man 2,3 Idiots
user 1,0.000000,0.0,0.000000,0.0
user 2,0.000000,0.5,0.000000,0.0
user 3,0.000000,0.5,0.736842,0.0
user 4,0.179487,0.0,0.589744,0.0


In [16]:
movies = ["Titanic","Martian","GOTG Vol 2"]
action = [1,0,1]
scifi = [1,1,1]
adventure = [0,1,1]
comedy = [0,0,1]
drama = [0,1,0]

df_item_features_new = pd.DataFrame({
    'movie':movies,
    'Action':action,
    'Sci-Fi':scifi,
    'Adventure':adventure,
    'Comedy':comedy,
    'Drama':drama
})

df_item_features_new


,movie,Action,Sci-Fi,Adventure,Comedy,Drama
0,Titanic,1,1,0,0,0
1,Martian,0,1,1,0,1
2,GOTG Vol 2,1,1,1,1,0


In [17]:
arr_item_features_new = np.array(df_item_features_new.drop('movie', axis = 1))

n_item_new = df_item_features_new.shape[0]

arr_user_items_score_new = np.empty((n_user,n_item_new))

for i in range(0, n_user):
  user_item_score = np.matmul(arr_item_features_new,arr_user_feature[i,:])
  arr_user_items_score_new[i,:] = user_item_score

df_user_items_score_new = pd.DataFrame(arr_user_items_score_new)
df_user_items_score_new.index = user
df_user_items_score_new.columns = df_item_features_new['movie']
df_user_items_score_new # score rekomendasi


movie,Titanic,Martian,GOTG Vol 2
user 1,0.428571,0.648352,0.802198
user 2,0.560000,0.500000,0.900000
user 3,0.473684,0.500000,0.736842
user 4,0.179487,0.769231,0.589744


## Urutan Rekomendasi
- user 1: GOTG
- user 2: GOTG
- user 3: GOTG
- user 4: Martian